In [1]:
from helper_preprocessing import *

path_tests = 'C:/Users/Simone/Desktop/Utilità tesi magistrale/data/tests.txt'
path_species = 'C:/Users/Simone/Desktop/Utilità tesi magistrale/data/species.txt'
path_results = 'C:/Users/Simone/Desktop/Utilità tesi magistrale/data/results.txt'


tests, species, results = load_raw_data(path_tests, path_results, path_species)

prefiltered_data = prefilter(species, tests, results)

del tests, species, results

db = select_impute_features(prefiltered_data)
db = repeated_experiments(db)
db['test_cas'] = db.test_cas.apply(to_cas)

db.to_csv('repeated_experiments.csv')

chemicals = extract_chemical_data(db)

print('Cleaning Chemical data...', ctime())
chemicals = process_smiles_features(chemicals)

print('The extracted chemicals are:', chemicals.shape[0])

chemicals.to_csv('chemicals.csv')

print('Merging chemical descriptors with experiments')
final_db = db.merge(chemicals, right_on = 'test_cas', left_on = 'test_cas')

print('Dimension of final dataset:', final_db.shape[0], 'experiments and', final_db.shape[1], 'features')
print('Final features:', [i for i in final_db.columns])

final_db.to_csv('lc_db_processed.csv')

Start! Sun Jan 31 20:16:05 2021
Loading raw data
tests table loaded!
species table loaded!
results table loaded!
Tests table dimensions:  (681605, 122)
Species table dimensions:  (27125, 15)
Results table dimensions:  (1006972, 137)
Prefiltering Step... Sun Jan 31 20:17:47 2021
Dimensions after prefiltering step:  (64341, 272)
Selection and Imputation Step... Sun Jan 31 20:17:56 2021
Features:
obs_duration_mean, obs_duration_unit, endpoint, effect, measurement, conc1_type, conc1_mean, conc1_unit, test_cas, test_location, exposure_type, control_type, media_type, application_freq_unit, class, tax_order, family, genus, species, 
Dimension after imputation step: (50160, 16)
Aggregation of repeated experiments Sun Jan 31 20:17:57 2021
Dimension after repeated experiments aggregation: (28815, 14)
Remark: Stable internet connection required for smiles and pubchem extraction!
Smiles extraction... Sun Jan 31 20:17:57 2021
Pubchem2d extraction... Sun Jan 31 20:45:33 2021
Molecular Descriptors ex

In [1]:
from helper_preprocessing import *

path_tests = 'C:/Users/Simone/Desktop/Utilità tesi magistrale/data/tests.txt'
path_species = 'C:/Users/Simone/Desktop/Utilità tesi magistrale/data/species.txt'
path_results = 'C:/Users/Simone/Desktop/Utilità tesi magistrale/data/results.txt'


tests, species, results = load_raw_data(path_tests, path_results, path_species)

prefiltered_data = prefilter(species, tests, results)

del tests, species, results

Start! Thu Apr 29 19:28:15 2021
Loading raw data
tests table loaded!
species table loaded!
results table loaded!
Tests table dimensions:  (681605, 122)
Species table dimensions:  (27125, 15)
Results table dimensions:  (1006972, 137)
Prefiltering Step... Thu Apr 29 19:29:59 2021
Dimensions after prefiltering step:  (64341, 272)


In [2]:
keep_columns = ['obs_duration_mean', 'obs_duration_unit',
       'endpoint', 'effect', 'measurement', 'conc1_type', 'conc1_mean',
       'conc1_unit', 'test_cas', 'test_location', 'exposure_type',
       'control_type', 'media_type', 'application_freq_unit', 'class',
       'tax_order', 'family', 'genus', 'species']
    
db = prefiltered_data.copy()
db = db[keep_columns]

del prefiltered_data

In [6]:
db.conc1_unit.value_counts()[:15]

ug/L       38076
mg/L        8758
ppm         5336
AI ug/L     4598
AI mg/L     2680
ppb         1423
ul/L        1204
AI ppm       319
uM           309
AI ng/L      234
AI ppb       228
%            162
mg/dm3       139
ml/L         134
umol/L       128
Name: conc1_unit, dtype: int64

In [7]:
def impute_conc(results_prefiltered):
    
    db = results_prefiltered.copy()
    
    to_drop_conc_mean = db[db.conc1_mean == 'NR'].index
    db_filtered_mean = db.drop(index = to_drop_conc_mean).copy()

    db_filtered_mean.loc[:,'conc1_mean'] = db_filtered_mean.conc1_mean\
                                                        .apply(lambda x: x.replace("*", "") if "*" in x else x).copy()

    to_drop_invalid_conc = db_filtered_mean[db_filtered_mean.conc1_mean == '>100000'].index
    db_filtered_mean.drop(index = to_drop_invalid_conc, inplace = True)

    db_filtered_mean.loc[:,'conc1_mean'] = db_filtered_mean.conc1_mean.astype(float).copy()

    to_drop_useless = db_filtered_mean[db_filtered_mean.conc1_mean == 0].index
    db_filtered_mean.drop(index = to_drop_useless, inplace = True)

    db_filtered_mean.loc[:,'conc1_unit'] = db_filtered_mean.conc1_unit\
                                                        .apply(lambda x: x.replace("AI ", "") if 'AI' in x else x)
    
    db_filtered_mean.loc[(db_filtered_mean.conc1_unit == 'ppb') | (db_filtered_mean.conc1_unit == 'ug/L'), 'conc1_mean'] = \
                db_filtered_mean.conc1_mean[(db_filtered_mean.conc1_unit == 'ppb') | (db_filtered_mean.conc1_unit == 'ug/L')]/1000
    
    db_filtered_mean.loc[db_filtered_mean.conc1_unit == 'ng/L', 'conc1_mean'] = db_filtered_mean.conc1_mean[db_filtered_mean.conc1_unit == 'ng/L']*(10**(-6))

    to_drop_unit = db_filtered_mean.loc[(db_filtered_mean.conc1_unit == 'uM') | (db_filtered_mean.conc1_unit == 'ul/L')].index
    db_filtered_mean.drop(index = to_drop_unit, inplace = True)

    to_drop_type = db_filtered_mean.loc[(db_filtered_mean.conc1_type == 'NC') |(db_filtered_mean.conc1_type == 'NR')].index
    db_filtered_mean.drop(index = to_drop_type, inplace = True)
    
    return db_filtered_mean

db = impute_conc(db)

In [10]:
cont = db.conc1_unit.value_counts()
# cont
sum(cont[~cont.index.isin(['mg/L', 'ppb', 'ppm', 'ug/L', 'ng/L'])])

1061

In [11]:
db.shape

(58438, 19)